# Task 1

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score

## Parte 1

In [32]:
## Se anadira droutout a la arquitectura LeNet-5 para mejorar la generalizacion del modelo. 
# Definir la arquitectura LeNet-5 con Dropout
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.dropout = nn.Dropout(0.5)  # Añadir Dropout con un 50% de probabilidad

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 16*5*5)
        x = self.dropout(torch.relu(self.fc1(x)))  # Aplicar Dropout después de la primera capa FC
        x = self.dropout(torch.relu(self.fc2(x)))  # Aplicar Dropout después de la segunda capa FC
        x = self.fc3(x)
        return x

In [33]:
# Cargar el dataset MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

In [34]:
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

In [35]:
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

In [36]:
# Inicializar el modelo, la función de pérdida y el optimizador
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LeNet5().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [37]:
## comprobar si se puede usar cuda con gpu. 
print(device)
print(torch.cuda.is_available())

cuda
True


In [38]:
# Entrenamiento
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    model.train()  # Asegurarse de que el modelo esté en modo de entrenamiento (Dropout activo)
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")

Epoch 1, Loss: 0.359000263984253
Epoch 2, Loss: 0.13579924102630467
Epoch 3, Loss: 0.10573464371139804
Epoch 4, Loss: 0.08930337681182039
Epoch 5, Loss: 0.07761206855581453
Epoch 6, Loss: 0.07075405709274735
Epoch 7, Loss: 0.06550046803742492
Epoch 8, Loss: 0.06018247390315325
Epoch 9, Loss: 0.05528907021235403
Epoch 10, Loss: 0.05188241279313224


In [39]:
# Función para calcular la métrica de precisión
def calculate_accuracy(model, dataloader, device):
    model.eval()  # Poner el modelo en modo evaluación (Dropout desactivado)
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_predictions)
    return accuracy

In [40]:
# Calcular la precisión en el conjunto de prueba
accuracy = calculate_accuracy(model, testloader, device)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 98.94%


## Parte 2

In [42]:
# Definir la arquitectura AlexNet
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 10),  # CIFAR10 tiene 10 clases
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x

In [43]:
# Cargar el dataset CIFAR10
transform = transforms.Compose([transforms.Resize(224),  # AlexNet necesita imágenes de mayor tamaño
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [44]:
trainset = torchvision.datasets.CIFAR10(root='./data2', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

100.0%


Extracting ./data2\cifar-10-python.tar.gz to ./data2


In [45]:
testset = torchvision.datasets.CIFAR10(root='./data2', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

Files already downloaded and verified


In [46]:
# Inicializar el modelo, la función de pérdida y el optimizador
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AlexNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [47]:
# Entrenamiento
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    model.train()
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")

Epoch 1, Loss: 1.8205744376643225
Epoch 2, Loss: 1.5468569516525463
Epoch 3, Loss: 1.4416546739223135
Epoch 4, Loss: 1.375955625710698
Epoch 5, Loss: 1.3253945802467721
Epoch 6, Loss: 1.2890181113952104
Epoch 7, Loss: 1.2496006385454823
Epoch 8, Loss: 1.2279362307476525
Epoch 9, Loss: 1.2091466494271639
Epoch 10, Loss: 1.1734921619515348


In [48]:
# Calcular la precisión en el conjunto de prueba
accuracy = calculate_accuracy(model, testloader, device)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 58.91%
